In [1]:
!pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [24]:
import os

os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "This form the Azure portal - model deployment"
os.environ["AZURE_OPENAI_API_KEY"] = 'This form the Azure portal - model deployment'


In [25]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [26]:
llm = AzureChatOpenAI(
            temperature=1,
            azure_deployment="gpt-4o",
        )

## Chain 1: Synopsis Chain.

In [38]:
template = """
  You are a playwright. Given the title of play and the era it is set in,
  it is your job to write a short synopsis for that title.

  Title: {title}
  Era: {era}

  Playwright: This is a synopsis for the above play:
"""

prompt_tempalte = PromptTemplate(
    input_variables=['title', 'era'],
    template=template
)

In [39]:
synopsis_chain = LLMChain(llm=llm,
                          prompt=prompt_tempalte,
                          output_key = 'synopsis')

## Chain 2: Review Chain.

In [40]:
template = """
  You are a play critic from the New York Times. Given the synopsis of play,
  it is your job to write a short review for that play.

  Play Synopsis:
  {synopsis}

  Review from a New York Times play critic of the above play:
"""

prompt_tempalte = PromptTemplate(
    input_variables=['synopsis'],
    template=template
)

In [30]:
review_chain = LLMChain(llm=llm,
                        prompt=prompt_tempalte,
                        output_key='review')


## Chain 3: Social Media Chain.

In [41]:
template = """
  You are a social media manager for a theater company.  Given the title of play,
  the era it is set in, the date,time and location, the synopsis of the play, and the review of the play,
  it is your job to write a short social media post for that play.

  Here is some context about the time and location of the play:
  Date and Time: {time}
  Location: {location}

  Play Synopsis:
  {synopsis}

  Review from a New York Times play critic of the above play:
  {review}

  Social Media Post:
"""


prompt_template = PromptTemplate(
    input_variables=["synopsis", "review", "time", "location"],
    template=template
                                 )


In [42]:
social_chain = LLMChain(llm=llm,
                        prompt=prompt_template,
                        output_key="social_post_text")

# Sequential Chain

In [45]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

In [46]:
overall_chain = SequentialChain(
    memory=SimpleMemory(
        memories={
            "time": "December 25th, 8pm PST",
            "location": "Theater in the Park"
            }
        ),
    chains = [synopsis_chain, review_chain, social_chain],
    input_variables=['title', 'era'],
    output_variables = ['social_post_text'],
    verbose = True
)

In [47]:
overall_chain.invoke({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': '🌅✨ Witness a heartwrenching tale of forbidden love and societal constraint enveloped in the serene, yet melancholy, coastal ambiance of Victorian-era Brighton. \n\n🖼️ Join us on **December 25th at 8pm PST** at the iconic **Theater in the Park** for "Tragedy at Sunset on the Beach." \n\n🎭 Follow the poignant journey of Cecilia Harrington, torn between duty to her family and her love for the impassioned artist Edward Mansfield. As the sun sets on their bittersweet romance, will they defy societal expectations or be forever imprisoned by them?\n\n🌊 "Tragedy at Sunset on the Beach" is a testament to the complexities of the human heart, the clash between duty and desire, and the inevitable tragedy of suppressed passions. 🌹\n\n✨ Don\'t miss this evocative production hailed as a must-see by critics!\n\n🎟️ #TheaterInThePark #Victo